In [1]:
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import random

# =========================
# Configuration
# =========================
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

# ✅ Nitter search URL for "depression"
nitter_url = "https://nitter.net/search?f=tweets&q=depression"

# =========================
# Open Nitter and go to search page
# =========================
driver.get("https://nitter.net/")
time.sleep(5)

driver.get(nitter_url)
time.sleep(5)

# =========================
# Data Storage
# =========================
tweets = []
unique_posts = set()
tweets_to_scrape = 40

# =========================
# Start Scraping
# =========================
while len(tweets) < tweets_to_scrape:
    # Scroll more aggressively but safely
    for _ in range(3):
        driver.execute_script("window.scrollBy(0, 2000);")
        time.sleep(random.uniform(1.5, 2.5))

    try:
        tweet_containers = WebDriverWait(driver, 5).until(
            EC.presence_of_all_elements_located((By.XPATH, '/html/body/div/div/div[2]/div[2]/div'))
        )
    except Exception as e:
        print("Tweet containers not found. Retrying...", e)
        continue

    print(f"🔍 Found {len(tweet_containers)} tweet containers.")

    for container in tweet_containers:
        if len(tweets) >= tweets_to_scrape:
            break

        try:
            username = container.find_element(By.XPATH, './/a[contains(@class, "fullname")]').text.strip()
        except:
            username = 'Unknown'

        try:
            content = container.find_element(By.XPATH, './/div[contains(@class, "tweet-content media-body")]').text.strip()
        except:
            content = 'No content'

        try:
            post_url = container.find_element(By.XPATH, './/div[contains(@class, "tweet-body")]//a[contains(@href, "/status/")]').get_attribute("href")
        except:
            post_url = 'No URL'

        try:
            stats_element = container.find_element(By.XPATH, './div[4]')
            comment_nb = stats_element.find_element(By.XPATH, './span[1]').text.strip()
            retweet_nb = stats_element.find_element(By.XPATH, './span[2]').text.strip()
            like_nb = stats_element.find_element(By.XPATH, './span[4]').text.strip()
        except:
            comment_nb = '0'
            retweet_nb = '0'
            like_nb = '0'

        try:
            like_val = int(like_nb.replace(',', '').strip())
        except:
            like_val = 0

        # Only add tweets with more than 1 like
        if like_val <= 1:
            continue

        if content not in unique_posts:
            unique_posts.add(content)
            tweets.append({
                'username': username,
                'content': content,
                'post_url': post_url,
                'comment_count': comment_nb,
                'retweet_count': retweet_nb,
                'like_count': like_val
            })
            print(f"✅ Scraped: {username} → {content[:30]}... ({len(tweets)}/{tweets_to_scrape})")

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(random.uniform(2, 3))

# =========================
# Save Final Dataset
# =========================
df = pd.DataFrame(tweets)
df.to_csv('depression_nitter_40.csv', index=False, encoding='utf-8')
print(f"✅ Finished: Saved {len(tweets)} tweets to depression_nitter_40.csv")

driver.quit()


🔍 Found 1 tweet containers.
✅ Scraped: TreVeyon Henderson → Breaks my heart seeing this. T... (1/40)
🔍 Found 1 tweet containers.
🔍 Found 1 tweet containers.
🔍 Found 1 tweet containers.
🔍 Found 1 tweet containers.
✅ Scraped: Ayesha → Abuse
Miscarriage
Pregnancy
Lo... (2/40)
🔍 Found 1 tweet containers.
✅ Scraped: 🏛 🌹PeriklestheGREAT 🌹 🏛 "Vox Populi, Vox Dei" → 🔥Woke Doctors said they had to... (3/40)
🔍 Found 1 tweet containers.
🔍 Found 1 tweet containers.
🔍 Found 1 tweet containers.
🔍 Found 1 tweet containers.
🔍 Found 1 tweet containers.
🔍 Found 1 tweet containers.
🔍 Found 1 tweet containers.
🔍 Found 1 tweet containers.
🔍 Found 1 tweet containers.
🔍 Found 1 tweet containers.
🔍 Found 1 tweet containers.
🔍 Found 1 tweet containers.
🔍 Found 1 tweet containers.
🔍 Found 1 tweet containers.
🔍 Found 1 tweet containers.
🔍 Found 1 tweet containers.
🔍 Found 1 tweet containers.
🔍 Found 1 tweet containers.
✅ Scraped: Quise 👻🩸 → If u ever been through some sh... (4/40)
🔍 Found 1 tweet containers.
🔍 F